In [2]:
import tensorflow as tf
import numpy as np

In [27]:
emb_encoder=a1
print(emb_encoder.shape.as_list())


encoder_outputs=[emb_encoder]
PAD_emb=tf.nn.embedding_lookup(emb,[1])

for enc_layer in range(6):
    with tf.variable_scope("encoder_%d"%enc_layer):
        emb_encoder=tf.pad(emb_encoder,paddings=[[0,0],[1,1],[0,0]],constant_values=PAD_emb)
        print(emb_encoder.shape.as_list())
        encoder_shape=emb_encoder.shape.as_list()

        emb_encoder=tf.reshape(emb_encoder,shape=[encoder_shape[0],encoder_shape[1],encoder_shape[2],1])# batch_size,seq_len,emb_dim,1

        filtersize=[3,10,1,20]
        filter=tf.Variable(initial_value=tf.truncated_normal(filtersize,stddev=0.01),name="filter")
        
        emb_encoder=tf.nn.conv2d(emb_encoder,filter=filter,strides=[1,1,1,1],padding="VALID",dilations=[1,1,1,1])
        emb_encoder=tf.reshape(emb_encoder,shape=[6,-1,10*2])
        A,B=tf.split(emb_encoder,2,axis=2)
        attn=tf.multiply(A,tf.nn.softmax(B))

        print(attn.shape.as_list())

        emb_encoder=attn+encoder_outputs[-1]
        a=tf.nn.softmax(emb_encoder,axis=1)
        print("a",a.shape.as_list())
        encoder_outputs.append(emb_encoder)
        print(emb_encoder.shape.as_list())



[6, 80, 10]
[6, 82, 10]
[6, 80, 10]
a [6, 80, 10]
[6, 80, 10]
[6, 82, 10]
[6, 80, 10]
a [6, 80, 10]
[6, 80, 10]
[6, 82, 10]
[6, 80, 10]
a [6, 80, 10]
[6, 80, 10]
[6, 82, 10]
[6, 80, 10]
a [6, 80, 10]
[6, 80, 10]
[6, 82, 10]


[6, 80, 10]
a [6, 80, 10]
[6, 80, 10]
[6, 82, 10]
[6, 80, 10]
a [6, 80, 10]
[6, 80, 10]


In [6]:
a=np.array(range(48))
a=a.reshape([6,8])
a=tf.constant(a,dtype=tf.float32)
b=tf.constant([1,0,1,1,0,0],dtype=tf.float32,shape=[6,1])
g=tf.constant([[1,0,1,1,0,0],[1,0,1,1,0,0]],dtype=tf.float32,shape=[2,6])

c=tf.multiply(b,a)
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)
d,f=sess.run([c,tf.nn.l2_normalize(g,1)])
print(d)
print(f)

[[  0.   1.   2.   3.   4.   5.   6.   7.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [ 16.  17.  18.  19.  20.  21.  22.  23.]
 [ 24.  25.  26.  27.  28.  29.  30.  31.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]]
[[ 0.57735026  0.          0.57735026  0.57735026  0.          0.        ]
 [ 0.57735026  0.          0.57735026  0.57735026  0.          0.        ]]


In [10]:
import tensorflow as tf 
a = tf.constant(1)
n = tf.constant(10)

def cond(a, n):
    return  a< n
def body(a,n):
    a = a + 1
    return a,n

a, n = tf.while_loop(cond, body, [a, n])
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    res = sess.run([a,n])
    print(res)


[10, 10]


In [ ]:

a=tf.constant([[123,34],[12,45],[56,23],[23,45],[65,13]],dtype=tf.int32)
b=tf.constant([[5],[7],[3],[1],[6]],tf.int32)
tf.global_variables_initializer()
sess=tf.Session()
v=sess.run([a*b])
print(v)


[array([[615, 170],
       [ 84, 315],
       [168,  69],
       [ 23,  45],
       [390,  78]], dtype=int32)]


In [7]:
class A():
    def __init__(self,a,b):
        self.a=a
        self.b=b
    
    def __call__(self, rouge):
        if "a"==rouge:
            return self.a+self.b
        elif "b" ==rouge:
            return self.a*self.b

a=A(3,2)
print(a("a"))


5


In [43]:
a=tf.Variable([])
table = tf.TensorArray(tf.int32, 12, clear_after_read=False, element_shape=[])
table=table.write(1,5)
table = tf.reshape(table.stack(), [3, 4])
print(table)
value=table[0,1]
value=tf.reshape(value,shape=[1])
value=tf.cast(value,tf.float32)
print(value)
a=tf.concat([a,value],0)
a=tf.concat([a,value],0)
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)
print(sess.run(a))

Tensor("Reshape_17:0", shape=(3, 4), dtype=int32)
Tensor("Cast_7:0", shape=(1,), dtype=float32)
[5. 5.]


In [5]:
#词汇表中对应的主题词
a=tf.constant([[11,1],[12,2],[13,3],[14,4],[15,5],[16,6]],dtype=tf.int32)
# 词汇表
# b=tf.constant([[1,11,13],[3,16,6],[5,6,2]],dtype=tf.int32)
b=tf.constant([[0,13],[1,16],[0,8]],dtype=tf.int32)

emb=tf.get_variable(name='embedding',shape=[20,5],dtype=tf.float32)
topic_emb=tf.get_variable(name='topics',shape=[10,5],dtype=tf.float32)

In [7]:
a_shape=a.get_shape()[0]
b_shape=b.get_shape()
size=b_shape[0]*b_shape[1]

def look_step(g,index,single,k1):
    t=tf.cond(tf.equal(single,a[k1,0]),
            lambda :(True,k1),
            lambda :(False,-1))
    return tf.logical_or(g,t[0]),tf.maximum(index,t[1]),single,k1+1
    
def _look(single):
    g=False
    k1=0
    index=-1
    g,index,*_=tf.while_loop(
        cond=lambda g,index,single,k1:k1<a_shape,
        body=look_step,
        loop_vars=[g,index,single,k1]
    )
    return g,index

k=tf.constant(1)
def loop_step(k,topic,matrix):
    g,index=_look(topic[k])
    matr=tf.cond(g,
                lambda :tf.gather(topic_emb,[a[index,1]]),
                lambda :tf.gather(emb,[topic[k]]))
    matr=tf.reshape(matr,[1,1,-1])
    matrix=tf.concat([matrix,matr],0)
    print(matrix)
    return k+1,topic,matrix

g=tf.reshape(b,[-1])
# matrix=tf.Variable([[]])
matrix=tf.nn.embedding_lookup(emb,[0])
matrix=tf.reshape(matrix,[1,1,-1])
_,_,matrix=tf.while_loop(
    cond=lambda k,b,*_:k<size,
    body=loop_step,
    loop_vars=[k,g,matrix],
    shape_invariants=[k.get_shape(),tf.TensorShape([None]),tf.TensorShape([None,1,5])]

)

matrix=tf.reshape(matrix,shape=[-1,b_shape[1],5])
print(matrix)
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)

matrix,topics,emm=sess.run([matrix,topic_emb,emb])
print(np.array(matrix).shape)
print(matrix)

print('------')
print(topics)
print('------')
print(emm)

Tensor("while_1/concat:0", shape=(?, 1, 5), dtype=float32)
Tensor("Reshape_5:0", shape=(?, 2, 5), dtype=float32)
(3, 2, 5)
[[[ 0.13084796 -0.3908043   0.19348922 -0.24810493  0.44234452]
  [ 0.05184722 -0.04054689  0.41993362  0.5034165   0.25516206]]

 [[-0.15891024 -0.00476465 -0.01022813  0.37674114 -0.00997618]
  [ 0.2298823  -0.06199652 -0.4847191   0.60867935 -0.6107369 ]]

 [[ 0.13084796 -0.3908043   0.19348922 -0.24810493  0.44234452]
  [-0.04860163  0.48754075 -0.3807378  -0.12713578  0.21223983]]]
------
[[ 0.21192467  0.04543304  0.36380446  0.31198138 -0.59245765]
 [ 0.50931627  0.48423976  0.6293971  -0.51611584 -0.31456184]
 [-0.17607754 -0.3145715   0.45998472 -0.23318592 -0.01605123]
 [ 0.05184722 -0.04054689  0.41993362  0.5034165   0.25516206]
 [ 0.04833382  0.3463604   0.4591692   0.29215127  0.6235923 ]
 [ 0.4013316   0.16012675  0.03698754 -0.37010384 -0.35769206]
 [ 0.2298823  -0.06199652 -0.4847191   0.60867935 -0.6107369 ]
 [-0.05478263  0.448682    0.13338143  

In [46]:
i = tf.constant(0)
l = tf.Variable([])
array = tf.Variable(tf.random_normal([10]))
def body(i, l):                                               
    temp = tf.gather(array,i)
    l = tf.concat([l, [temp]], 0)
    return i+2, l
def cond(i,l):
   return i < 10
index, list_vals = tf.while_loop(cond, body, [i, l],shape_invariants=[i.get_shape(),
                                                   tf.TensorShape([None])])
sess = tf.Session()
sess.run(tf.global_variables_initializer())
a1,b1=sess.run([list_vals,array])
print(a1)
print(b1)

[-0.12611371  0.08051919  2.6596558  -2.5261319   1.4858145 ]
[-0.12611371 -1.0288202   0.08051919 -0.40016657  2.6596558   0.4868604
 -2.5261319   0.5280155   1.4858145  -0.3094576 ]


In [47]:
gh1=tf.gather(emb,[1])
gh2=tf.gather(emb,[0])
gh=tf.concat([gh2,gh1],axis=0)
print(gh)
a1,b1=sess.run([emb,gh])

Tensor("concat_18:0", shape=(2, 5), dtype=float32)


In [20]:
t = tf.constant([[[1, 1, 1], [2, 2, 2], [7, 7, 7]],
                 [[3, 3, 3], [4, 4, 4], [8, 8, 8]],
                 [[5, 5, 5], [6, 12, 6], [9, 13, 9]],
                 [[5,12, 5], [6, 6, 6], [9, 9, 9]]])

# z1 = tf.strided_slice(t, [1], [-1], [1])
z2 = tf.strided_slice(t, [0, 2], [4, 3], [1, 1])
z3 = tf.strided_slice(t, [1, 0, 1], [-1, 2, 3], [1, 1, 1])
z4=tf.expand_dims(z2[:,-1,:],1)
print(t)
print(z4)
with tf.Session() as sess:
    # print(sess.run(z1))
    print(sess.run(z2))
    print(sess.run(z4))


Tensor("Const_17:0", shape=(4, 3, 3), dtype=int32)
Tensor("ExpandDims:0", shape=(4, 1, 3), dtype=int32)
[[[ 7  7  7]]

 [[ 8  8  8]]

 [[ 9 13  9]]

 [[ 9  9  9]]]
[[[ 7  7  7]]

 [[ 8  8  8]]

 [[ 9 13  9]]

 [[ 9  9  9]]]


In [26]:
s=np.array(range(60))

a=tf.constant(s,dtype=tf.float32,shape=[3,4,5])
c=tf.reduce_max(a,-1)
print(c)
d=tf.argmax(a,-1)
init=tf.global_variables_initializer()
sess=tf.Session()
print(sess.run([c,d]))


Tensor("Max_1:0", shape=(3, 4), dtype=float32)
[array([[ 4.,  9., 14., 19.],
       [24., 29., 34., 39.],
       [44., 49., 54., 59.]], dtype=float32), array([[4, 4, 4, 4],
       [4, 4, 4, 4],
       [4, 4, 4, 4]])]


In [3]:
i0 = tf.constant(0)
m0 = tf.ones([3, 1])
# x = tf.get_variable('www', shape=(3,1), initializer=tf.zeros_initializer)
# loop = 5
# def _cond(i0, m0):
#   return tf.less(i0, loop-1)
# 
# def _res(i0, m0):
#   n = tf.ones([3, 1]) + x
#   m0 = tf.concat([m0, n], axis=0)
#   return i0+1, m0

# loop_vars=[tf.constant(0,tf.int32),tf.TensorArray(m0.dtype,size=loop)]
# i0, m0 = tf.while_loop(
#     _cond, _res, loop_vars=[i0, m0],
#     shape_invariants=[i0.get_shape(), tf.TensorShape([None, 1])])
# 
# m0.set_shape([loop*3,1])
# opt = tf.train.AdagradOptimizer(1)
# 
# grad = opt.compute_gradients(m0)
# sess.run(tf.global_variables_initializer())
# 
# print(sess.run(grad))

def mystack(x, n):
    loop_vars = [
        tf.constant(0, tf.int32),
        tf.TensorArray(x.dtype, size=n),
    ]
    _, fx = tf.while_loop(
        lambda j, _: j < n,
        lambda j, result: (j + 1, result.write(j, 1+x)),
        loop_vars= loop_vars
    )
    return tf.reshape(fx.stack(), (-1,1) )
 
x = tf.constant(np.random.randn(3,1), tf.float32 )
print(x)
loop = 5
m = mystack(x,loop)
print(m)
 
with tf.Session() as sess:
    print(sess.run([m,x]))

Tensor("Const_1:0", shape=(3, 1), dtype=float32)
Tensor("Reshape:0", shape=(?, 1), dtype=float32)
[array([[0.9740323 ],
       [2.1492686 ],
       [0.22505063],
       [0.9740323 ],
       [2.1492686 ],
       [0.22505063],
       [0.9740323 ],
       [2.1492686 ],
       [0.22505063],
       [0.9740323 ],
       [2.1492686 ],
       [0.22505063],
       [0.9740323 ],
       [2.1492686 ],
       [0.22505063]], dtype=float32), array([[-0.02596773],
       [ 1.1492685 ],
       [-0.7749494 ]], dtype=float32)]


In [3]:
import tensorflow as tf
# 22 scope (name_scope/variable_scope)
from __future__ import print_function
 
class TrainConfig:
    batch_size = 20
    time_steps = 20
    input_size = 10
    output_size = 2
    cell_size = 11
    learning_rate = 0.01
 
 
class TestConfig(TrainConfig):
    time_steps = 1
 
 
class RNN(object):
 
    def __init__(self, config):
        self._batch_size = config.batch_size
        self._time_steps = config.time_steps
        self._input_size = config.input_size
        self._output_size = config.output_size
        self._cell_size = config.cell_size
        self._lr = config.learning_rate
        self._built_RNN()
 
    def _built_RNN(self):
        with tf.variable_scope('inputs'):
            self._xs = tf.placeholder(tf.float32, [self._batch_size, self._time_steps, self._input_size], name='xs')
            self._ys = tf.placeholder(tf.float32, [self._batch_size, self._time_steps, self._output_size], name='ys')
        with tf.name_scope('RNN'):
            with tf.variable_scope('input_layer'):
                l_in_x = tf.reshape(self._xs, [-1, self._input_size], name='2_2D')  # (batch*n_step, in_size)
                # Ws (in_size, cell_size)
                Wi = self._weight_variable([self._input_size, self._cell_size])
                print(Wi.name)
                # bs (cell_size, )
                bi = self._bias_variable([self._cell_size, ])
                # l_in_y = (batch * n_steps, cell_size)
                with tf.name_scope('Wx_plus_b'):
                    l_in_y = tf.matmul(l_in_x, Wi) + bi
                l_in_y = tf.reshape(l_in_y, [-1, self._time_steps, self._cell_size], name='2_3D')
 
            with tf.variable_scope('cell'):
                cell = tf.contrib.rnn.BasicLSTMCell(self._cell_size)
                with tf.name_scope('initial_state'):
                    self._cell_initial_state = cell.zero_state(self._batch_size, dtype=tf.float32)
 
                self.cell_outputs = []
                cell_state = self._cell_initial_state
                for t in range(self._time_steps):
                    if t > 0: tf.get_variable_scope().reuse_variables()
                    cell_output, cell_state = cell(l_in_y[:, t, :], cell_state)
                    self.cell_outputs.append(cell_output)
                self._cell_final_state = cell_state
 
            with tf.variable_scope('output_layer'):
                # cell_outputs_reshaped (BATCH*TIME_STEP, CELL_SIZE)
                cell_outputs_reshaped = tf.reshape(tf.concat(self.cell_outputs, 1), [-1, self._cell_size])
                Wo = self._weight_variable((self._cell_size, self._output_size))
                bo = self._bias_variable((self._output_size,))
                product = tf.matmul(cell_outputs_reshaped, Wo) + bo
                # _pred shape (batch*time_step, output_size)
                self._pred = tf.nn.relu(product)    # for displacement
 
        with tf.name_scope('cost'):
            _pred = tf.reshape(self._pred, [self._batch_size, self._time_steps, self._output_size])
            mse = self.ms_error(_pred, self._ys)
            mse_ave_across_batch = tf.reduce_mean(mse, 0)
            mse_sum_across_time = tf.reduce_sum(mse_ave_across_batch, 0)
            self._cost = mse_sum_across_time
            self._cost_ave_time = self._cost / self._time_steps
 
        with tf.variable_scope('trian'):
            self._lr = tf.convert_to_tensor(self._lr)
            self.train_op = tf.train.AdamOptimizer(self._lr).minimize(self._cost)
 
    @staticmethod
    def ms_error(y_target, y_pre):
        return tf.square(tf.subtract(y_target, y_pre))
 
    @staticmethod
    def _weight_variable(shape, name='weights'):
        initializer = tf.random_normal_initializer(mean=0., stddev=0.5, )
        return tf.get_variable(shape=shape, initializer=initializer, name=name)
 
    @staticmethod
    def _bias_variable(shape, name='biases'):
        initializer = tf.constant_initializer(0.1)
        return tf.get_variable(name=name, shape=shape, initializer=initializer)
 
 


In [8]:
if __name__ == '__main__':
    train_config = TrainConfig()  #定义train_config
    test_config = TestConfig()
 
#     # the wrong method to reuse parameters in train rnn
#     with tf.variable_scope('train_rnn'):
#         train_rnn1 = RNN(train_config)
#     with tf.variable_scope('test_rnn'):
#         test_rnn1 = RNN(test_config)
 
    # the right method to reuse parameters in train rnn
    #目的使train的RNN调用参数，然后利用variable_scope方法共享RNN,让test的RNN再次调用一样的参数，
    with tf.variable_scope('rnn12') as scope:
        
        train_rnn2 = RNN(train_config)
        scope.reuse_variables()        #告诉TF想重复利用RNN的参数
        test_rnn2 = RNN(test_config)
        sess = tf.Session()
        # tf.initialize_all_variables() no long valid from
        # 2017-03-02 if using tensorflow >= 0.12
        if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
            init = tf.initialize_all_variables()
        else:
            init = tf.global_variables_initializer()
        sess.run(init)


rnn12/input_layer/weights:0


rnn12/input_layer/weights:0
